<img src="img/datenschule_logo_web.png" width="25%"  align="left">

<img src="img/coat_of_arms_of_berlin.svg" width="11%"  align="right">
# Aktivitäten und Partnerschaften an Berliner Schulen
In diesem Notebook dokumnetieren wir unser Vorgehen bei der Auswertung der Partnerschaften und Aktivitäten <br> der Berliner Schulen. Da wir die Daten auf der Website  laufend aktualisieren, können die Ergebnisse hier <br> von denen auf der Website abweichen. Der Berechnungsweg ist allerdings der Gleiche.
 <br> Mehr Informationen über das Schulsystem in Deutschland gibt es auf <a href='https://jedeschule.de'>jedeschule.de<a>

In [2]:
#Für die Berechnung nutzen wir hier die Pandas-Library
import pandas as pd
import urllib.request, json 

In [3]:
with urllib.request.urlopen("https://lab.okfn.de/jedeschule/all_schools_geocoded.json") as url:
    data = json.loads(url.read().decode())
    berlin_data = [item for item in data if item['state'] == 'BE']
    partner_data = []
    wg_data = []
    for school in berlin_data:
        for partner in school['partner']:
            partner_data.append({
                "Schule": school['name'],
                "ID": school['id'],
                "school_type" : school['school_type'],
                "partner_name" : partner['name'],
                "Type_grob": partner['type']['grob'],
                "Type_mittel": partner['type']['mittel'],
                "Type_detail": partner['type']['detail']
            })
        for wg in school['programs']['working_groups']:
            wg_data.append({
                "id": school['id'],
                "ag_name" : wg['entity'],
                "ag_cat" : wg['category'],
                "school" : school['official_id']
            })

### 1. Auswertung der Schulaktivitäten in Berlin

Zunächst werden die Berliner Schulen von jedeschule.de geladen und für die Analyse vorbereitet

In [16]:
df_berlin_ag = pd.DataFrame.from_dict(wg_data)

In [17]:
df_berlin_ag.head(5)

,ag_cat,ag_name,id,school
0,Naturwissenschaft / Technik,Astronomie,BE-15586,04A06
1,Naturwissenschaft / Technik,Astrophysik,BE-15586,04A06
2,Literatur / Medien,Literatur,BE-15586,04A06
3,Kunst / Kultur,Bildende Kunst,BE-15586,04A06
4,Sport,Rudern,BE-15590,04P11


Zu Beginn einer Analyse ist es sinnvoll, einen Überblick über die Daten zu gewinnen.

In [18]:
df_berlin_ag.describe(include='all')

,ag_cat,ag_name,id,school
count,3853,3853,3853,3853
unique,10,562,492,469
top,Sport,Fußball,BE-16461,01P22
freq,1322,258,29,48


Mit dem Befehl *'describe'* können wir eine Zusammenfassung der Daten erhalten. Wir sehen: 
 * *unique*: 413 der 800 allgemeinbildenden Schulen in Berlin kommunizieren ihre Projekte und Aktivitäten Projekte und Aktivitäten an die Senatsverwaltung.
 * *count*: Es werden über 3300 Aktivitäten an Berliner Schulen angeboten.

In [19]:
# Anzahl der Schulen in Berlin
schools_count = len(df_berlin_ag.id.unique())

# Anzahl der Angebotenen AGs
projects_count = len(df_berlin_ag)

# Durchschnittliche Anzahl an Projekten an Beriner Schulen
projects_average = projects_count / schools_count
projects_average

7.83130081300813

Um zu errechnen wie viele Projekte eine Schule in Berlin durchschnittlich anbietet, teilen wir die Anzahl der Projekte durch die Anzahl der Schulen. Da die CSV-Datei für jedes Angebot einen eigenen Zeileneintrag enthält, werden auch Schulen doppelt genannt. Mit der Funktion *'unique()'* filtern wir doppelte Schuleinträge aus. Dadurch erhalten wir die Anzahl der Schulen in Berlin, die ihre Aktivitäten kommuniziert haben. <br>
**Ergebnis:**
* Durchschnittlich bietet eine Schule in Berlin ihren Schüler*innen rund 8 Aktivitäten an.

In [20]:
#Wie viel Projekte aus unterschiedlichen Bereichen bieten die Schulen ihren Schüler*innen?
category_unique = df_berlin_ag.drop_duplicates(subset=['id','ag_cat'])

#Durchschnittliche Anzahl von Aktivitätskategorien pro Schule
category_avg = len(category_unique) / schools_count
print('In Berlin werden in durchschnittlich %s Kategorien Aktivitäten angeboten.' %(round(category_avg,2)))

In Berlin werden in durchschnittlich 4.02 Kategorien Aktivitäten angeboten.


In [21]:
len(category_unique)

1977

In [23]:
categories_pivot = pd.pivot_table(category_unique,index=['ag_cat'],values='id',aggfunc='count')
df_categories = pd.DataFrame(categories_pivot)
df_categories['% Schulen mit Projekt-Kategorie'] = (df_categories.id / schools_count * 100).round(2)
df_categories.sort_values(by='id',ascending=False)

,id,% Schulen mit Projekt-Kategorie
ag_cat,,
Sport,423,85.98
Musik / Tanz,368,74.80
Kunst / Kultur,250,50.81
Naturwissenschaft / Technik,215,43.70
Handwerk,192,39.02
Literatur / Medien,179,36.38
Sprachen,126,25.61
Gesellschaft / Partizipation,113,22.97
Umwelt,91,18.50


Um zu errechen wie viele Schulen für eine jeweilige Kategorie Aktivitäten anbieten, nutzen wir die Pivot-Funktion. *(pivot_table())*. Mit der Funktion gruppieren wir die Aktivitäten nach ihren Kategorien. Teilt man diese Werte (hier unter Id) durch die Gesamtanzahl der Schulen, erhalten wir den prozentualen Anteil An Schulen die Aktivitäten in einer Kategorie anbieten.


### 2. Auswertung der Schulpartnerschaften in Berlin

In [24]:
df_berlin_partner = pd.DataFrame.from_dict(partner_data)

In [25]:
df_berlin_partner.head(5)

,ID,Schule,Type_detail,Type_grob,Type_mittel,partner_name,school_type
0,BE-15591,Nord-Grundschule,None,Unbestimmt,Modell/Förderprogramm/Projekt,Berliner Literaturinitiative (Projekt / Arbeit...,Grundschule
1,BE-15591,Nord-Grundschule,Polizei,Öffentliche Infrastruktur,Öffentliche Infrastruktur (Sicherheit/Ordnung/...,Berliner Polizei,Grundschule
2,BE-15591,Nord-Grundschule,None,Wirtschaftsakteur,Modell/Förderprogramm/Projekt,Computer Bildung Berlin (CBB),Grundschule
3,BE-15591,Nord-Grundschule,Verein,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Kaizen Berlin e.V. (Judo),Grundschule
4,BE-15591,Nord-Grundschule,Verein,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Projekt fee - freiwillig - engagiert - ehrenam...,Grundschule


Auch hier ist es zunächst sinnvoll,einen Blick in die Daten zu werfen.

In [26]:
df_berlin_partner.describe(include='all')

,ID,Schule,Type_detail,Type_grob,Type_mittel,partner_name,school_type
count,2977,2977,1595,2977,2977,2976,2977
unique,410,410,23,7,25,2122,15
top,BE-16610,Albrecht-von-Graefe-Schule (Integrierte Sekund...,Verein,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Verein Berliner Kaufleute und Industrieller (V...,Grundschule
freq,25,25,829,1118,999,29,1127


Über die *'describe()'* Funktion bekommen wir die Anzahl der Partnerschaften (2976) sowie die Anzahl der Schulen (411)

In [30]:
typ_partner = pd.DataFrame(df_berlin_partner.Type_grob.value_counts())
typ_partner = pd.DataFrame(pd.pivot_table(df_berlin_partner,index=['Type_grob'],values='partner_name', aggfunc='count'))
typ_partner['Verteilung'] = (typ_partner / sum(typ_partner.partner_name)).round(4)
typ_partner.sort_values(by='partner_name', ascending=False)

,partner_name,Verteilung
Type_grob,,
Gemeinnütziger Akteur,1118,0.3757
Öffentliche Infrastruktur,884,0.2970
Wirtschaftsakteur,533,0.1791
Partnerschule,246,0.0827
Unbestimmt,119,0.0400
Religiöse Einrichtung,42,0.0141
Verband / Kammer / Innung / Gewerkschaft,34,0.0114


In [31]:
schulkategorien = {'Grundschule' : [' Grundschule',
                                     'Grundschule'],
                    'Sekundarschule' : ['Integrierte Sekundarschule',
                                        ],
                    'Berufliche Schule' : ['Berufsfachschule',
                                           'Berufsoberschule',
                                           'Berufsschule',
                                           'Berufsschule mit sonderpäd. Aufgaben',
                                           'Fachoberschule',
                                           'Fachschule'],
                    'Förderschulen' : ['Förderschwerp."Geistige Entwicklung"',
                                       'Förderschwerp."Lernen"',
                                       'Übrige Förderschwerpunkte'
                                       ],
                    'Sonstige' :['Freie Waldorfschule',
                                 'Sonstige Ergänzungsschule',
                                 ],
                   
                   'Gymnasium' : ['Berufliches Gymnasium',
                                  'Gymnasien']
                   
                   }

Um die unterschiedlichen Schultypen zu vereinheitlichen, ordnen wir diese in 6 Kategorien ein: Grundschule, Sekundarschule, Berufliche Schule, Förderschulen, Gymnasium, Sonstige. Dazu erstellen wir das Dictionary: Schulkategorien.

In [32]:
def check_up_schultyp(schooltyp):
    for key, value in schulkategorien.items():
        for a in value:
            if a in str(schooltyp):
                return(key)

In [34]:
# Hier mappen wir die einzelnen Schultypen auf die 6 Schultypkategorien und säubern leere Zeilen aus.
df_berlin_partner['Schooltype_broad'] = df_berlin_partner.school_type.apply(check_up_schultyp)

In [85]:
df_berlin_partner.head(5)

,Schule,ID,school_type,Partner_name,Type_grob,Type_mittel,Type_detail,Schooltype_broad
0,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,AAP-Programm,Unbestimmt,Modell/Förderprogramm/Projekt,NaN,Gymnasium
1,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Arbeitskreis Medizintechnik Berlin-Brandenburg,Öffentliche Infrastruktur,Öffentliche Infrastruktur (Bildung/Uni/Hochsch...,NaN,Gymnasium
2,OSZ Handel 1,BE-16262,Berufliches Gymnasium,Ausbildungszentrum ROC Nijmegen,Öffentliche Infrastruktur,Familien-/Bildungs-/Kulturzentrum,NaN,Gymnasium
3,OSZ Informations- und Medizintechnik,BE-15754,Berufliches Gymnasium,Baumann Bildung & Qualifizierung (BBQ),Wirtschaftsakteur,Wirtschaftsakteur (Bildung),NaN,Gymnasium
4,Louise-Schroeder-Schule (OSZ Bürowirtschaft un...,BE-15687,Berufliches Gymnasium,Bildung für Berlin und Brandenburg e.V.,Gemeinnütziger Akteur,Gemeinnütziger Akteur,Verein,Gymnasium


In [35]:
#Durchscnittliche Anzahl an Partnerschaften pro Schule
berlin_unique = pd.DataFrame(df_berlin_partner.drop_duplicates())
len(berlin_unique) / len(berlin_unique.ID.unique())

7.258536585365854

In [41]:
partner_schulen = pd.DataFrame(pd.pivot_table(df_berlin_partner,index=['Schooltype_broad'],values='school_type',columns='Type_grob', aggfunc='count'))
partner_schulen = partner_schulen.fillna(0)

In [42]:
df_berlin_partner.head(2)

,ID,Schule,Type_detail,Type_grob,Type_mittel,partner_name,school_type,Schooltype_broad
0,BE-15591,Nord-Grundschule,None,Unbestimmt,Modell/Förderprogramm/Projekt,Berliner Literaturinitiative (Projekt / Arbeit...,Grundschule,Grundschule
1,BE-15591,Nord-Grundschule,Polizei,Öffentliche Infrastruktur,Öffentliche Infrastruktur (Sicherheit/Ordnung/...,Berliner Polizei,Grundschule,Grundschule


#### Wie viele Akteure für die jeweilige Kategorie kommen auf  eine Schule in Berlin?

In [43]:
dftyp = df_berlin_partner.drop_duplicates(subset=['ID','school_type'])
dftyp = pd.DataFrame(dftyp.Schooltype_broad.value_counts())

In [44]:
dftyp

,Schooltype_broad
Grundschule,164
Sekundarschule,98
Gymnasium,92
Berufliche Schule,36
Förderschulen,16
Sonstige,4


Hier errechnen wir die Anzahl der Schulen in Berlin für jeden Schultyp.

In [45]:
partner_schulen['Schulanzahl'] = dftyp

In [46]:
partner_schulen.head(2)

Type_grob,Gemeinnütziger Akteur,Partnerschule,Religiöse Einrichtung,Unbestimmt,Verband / Kammer / Innung / Gewerkschaft,Wirtschaftsakteur,Öffentliche Infrastruktur,Schulanzahl
Schooltype_broad,,,,,,,,
Berufliche Schule,49.0,9.0,4.0,20.0,10.0,55.0,43.0,36
Förderschulen,40.0,3.0,3.0,0.0,1.0,14.0,19.0,16


Anschließend fügen wir die Anzahl der Schulen pro Schultyp an das zuvorerstellte Dataframe mit der Anzahl der Partnerschaften pro Schultyp.

In [47]:
partner_schulen[['Gemeinnütziger Akteur','Partnerschule','Religiöse Einrichtung', 'Unbestimmt', 'Verband / Kammer / Innung / Gewerkschaft', 'Wirtschaftsakteur', 'Öffentliche Infrastruktur']].div(partner_schulen.Schulanzahl, axis=0)

Type_grob,Gemeinnütziger Akteur,Partnerschule,Religiöse Einrichtung,Unbestimmt,Verband / Kammer / Innung / Gewerkschaft,Wirtschaftsakteur,Öffentliche Infrastruktur
Schooltype_broad,,,,,,,
Berufliche Schule,1.361111,0.250000,0.111111,0.555556,0.277778,1.527778,1.194444
Förderschulen,2.500000,0.187500,0.187500,0.000000,0.062500,0.875000,1.187500
Grundschule,3.073171,0.164634,0.091463,0.231707,0.018293,0.792683,2.500000
Gymnasium,1.630435,1.826087,0.130435,0.293478,0.097826,1.347826,2.293478
Sekundarschule,3.806122,0.377551,0.071429,0.346939,0.112245,2.071429,2.040816
Sonstige,0.500000,0.500000,0.250000,0.000000,0.000000,1.750000,0.500000


Anschließend teilen wir die Anzahl der Partnerschaften pro Akteurskategorie durch die jeweilige Anzahl der Schulen pro Schultyp. Die obenstehende Tabelle zeigt also für jeden Schultyp wie viele Partnerschaften sie durchschnittlich pro Akteurskategorie eingeht.